<a href="https://colab.research.google.com/github/Olfeng-xalaz/Fallstudie_SCM/blob/main/Fallstudie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# pip als Paketmanager
! pip install -q pyscipopt
! pip install pandas

In [19]:
import pandas as pd
import math
from pyscipopt import Model, quicksum

# Optimierungsmodell zur Elektrifizierung der Logistik

### Import der CSV Dateien

In [20]:
import os

# Prüfen ob in Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    # In Colab: Repository klonen (nur einmal)
    if not os.path.exists('Fallstudie_SCM'):
        !git clone https://github.com/Olfeng-xalaz/Fallstudie_SCM.git
    folder = "Fallstudie_SCM/DataCSV"
else:
    # Lokal in VS Code: Relativer Pfad
    folder = os.path.join(os.path.dirname(__file__), "DataCSV") if "__file__" in dir() else "DataCSV"

In [21]:
chargers = pd.read_csv(f"{folder}/chargers.csv", sep=";")

In [22]:
chargers.head()

,charger_model,capex_yearly,opex_yearly,max_power,charging_spots
0,Alpitronic-50,3000,1000,50,2
1,Alpitronic-200,10000,1500,200,2
2,Alpitronic-400,16000,2000,400,2


In [23]:
dtrucks_specs = pd.read_csv(f"{folder}/diesel_trucks.csv", sep=";")

In [24]:
dtrucks_specs.head()

,truck_model,capex_yearly,opex_yearly,avg_diesel_per_100km,kfz_yearly,gross_vehicle_weight,emission_class,co2_emission_class
0,ActrosL,24000,6000,26,556,40,EURO 6,1


In [25]:
etrucks_specs = pd.read_csv(f"{folder}/electric_trucks.csv", sep=";")

In [26]:
etrucks_specs.head()

,truck_model,capex_yearly,opex_yearly,avg_energy_kWh_per_100km,thg_yearly,max_power,soc_max_kWh
0,eActros600,60000,6000,110,1000,400,621
1,eActros400,50000,5000,105,1000,400,414


In [27]:
routes = pd.read_csv(f"{folder}/routes.csv", sep=";")

In [28]:
routes.head()

,route_id,route_name,distance_total,distance_toll,starttime,endtime
0,t-4,Nahverkehr,250,150,06:45,17:15
1,t-5,Nahverkehr,250,150,06:30,17:00
2,t-6,Nahverkehr,250,150,06:00,16:30
3,s-1,Ditzingen,120,32,05:30,15:30
4,s-2,Ditzingen,120,32,06:00,16:00


### Indexmengen

In [29]:
#Erstellen einer Modellinstanz
scip = Model()

In [30]:
R = routes["route_id"].unique() # Menge der Routen
C = chargers["charger_model"].unique() # Menge der Charger-Modelle
L = pd.concat([dtrucks_specs["truck_model"],etrucks_specs["truck_model"]]).unique() # Menge der Fahrzeugmodelle
P = range(1, 21) # Potenzielle Fahrzeuge
T = range(1, 97) # 96 Zeitintervalle pro Tag
I = range(0, 4) # Potenzielle Säulen-Slots
# T_night = [t for t in T if t > 18*4 or t <= 6*4] # Nachtintervalle: 18:00–06:00


print("R (Routen):", R)
print("C (Charger):", C)
print("L (Fahrzeugmodelle):", L)
print("P (potenzielle Fahrzeuge):", list(P))
print("T (Zeitintervalle):", list(T))
print("I (Säulen):", list(I))
# print("T_night (Nachtintervalle):", list(T_night))


R (Routen): ['t-4' 't-5' 't-6' 's-1' 's-2' 's-3' 's-4' 'w1' 'w2' 'w3' 'w4' 'w5' 'w6'
 'w7' 'r1' 'r2' 'r3' 'h3' 'h4' 'k1']
C (Charger): ['Alpitronic-50' 'Alpitronic-200' 'Alpitronic-400']
L (Fahrzeugmodelle): ['ActrosL' 'eActros600' 'eActros400']
P (potenzielle Fahrzeuge): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
T (Zeitintervalle): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96]
I (Säulen): [0, 1, 2, 3]


### Parameter

In [31]:
# 3) Parameter aus routes.csv
# -----------------------------
dist_total = dict(zip(routes["route_id"], routes["distance_total"]))
dist_toll  = dict(zip(routes["route_id"], routes["distance_toll"]))
start_time = dict(zip(routes["route_id"], routes["starttime"]))
end_time   = dict(zip(routes["route_id"], routes["endtime"]))

# Hinweis:
# A[r,t] (binär: Tour r läuft im Intervall t) und g[r,t] (kWh-Verbrauch pro Intervall)
# hängen von Zeitdiskretisierung + Tourdauer ab.
# Das bauen wir später, sobald klar ist, wie starttime/endtime formatiert sind.

# -----------------------------
# 4) Parameter aus chargers.csv
# -----------------------------
capex_ch = dict(zip(chargers["charger_model"], chargers["capex_yearly"]))
opex_ch  = dict(zip(chargers["charger_model"], chargers["opex_yearly"]))
pmax_ch  = dict(zip(chargers["charger_model"], chargers["max_power"]))        # kW
spots_ch = dict(zip(chargers["charger_model"], chargers["charging_spots"]))   # Anzahl Ladepunkte

# -----------------------------
# 5) Parameter aus electric_trucks.csv
# -----------------------------
capex_veh = {}
opex_veh  = {}

cons_e = {}       # kWh/100km
thg_e  = {}       # €/a
pmax_veh = {}     # kW max Ladeleistung
batt_kwh = {}     # kWh Batterie (soc_max)

for _, row in etrucks_specs.iterrows():
    m = row["truck_model"]
    capex_veh[m] = row["capex_yearly"]
    opex_veh[m]  = row["opex_yearly"]
    cons_e[m]    = row["avg_energy_kWh_per_100km"]
    thg_e[m]     = row["thg_yearly"]
    pmax_veh[m]  = row["max_power"]
    batt_kwh[m]  = row["soc_max_kWh"]

# -----------------------------
# 6) Parameter aus diesel_trucks.csv
# -----------------------------
kfz_d = {}       # €/a
cons_d = {}      # ggf. l/100km (nur falls du Diesel-Kraftstoffkosten modellierst)

for _, row in dtrucks_specs.iterrows():
    m = row["truck_model"]
    capex_veh[m] = row["capex_yearly"]
    opex_veh[m]  = row["opex_yearly"]
    if "kfz_yearly" in dtrucks_specs.columns:
        kfz_d[m] = row["kfz_yearly"]
    if "avg_diesel_per_100km" in dtrucks_specs.columns:
        cons_d[m] = row["avg_diesel_per_100km"]

# -----------------------------
# 7) Abgeleitete Parameter: Energiebedarf pro Route und e-Lkw (E[r,e])
# -----------------------------
E_route_e = {}  # (r,e) -> kWh

for r in R:
    for e in cons_e.keys():
        E_route_e[(r, e)] = dist_total[r] * cons_e[e] / 100.0

# -----------------------------
# 8) Falltext-Parameter (Konstanten)
# -----------------------------
N_days = 260
delta_h = 0.25  # 15 Minuten = 0.25 Stunden

# Netz & Tarif
P_grid_max = 500.0
P_grid_add = 500.0
capex_grid_ext = 10000.0

c_energy = 0.25   # €/kWh
c_base = 1000.0   # €/a
c_peak = 150.0    # €/kW*a  <-- falls nicht gegeben: als Annahme setzen (hier Beispielwert)

# Maut
c_toll = 0.34      # €/km mautpflichtig

# Speicher
c_batP = 30.0      # €/kW*a
c_batE = 350.0     # €/kWh*a
bat_opex_rate = 0.02
roundtrip_eff = 0.98
eta = math.sqrt(roundtrip_eff)  # für lineare Lade/Entlade-Gleichungen
dod = 0.975
soc_bat_min_frac = 1.0 - dod    # = 0.025

# -----------------------------
# 9) Kurzer Test-Print der wichtigsten Parameter
# -----------------------------
print("Beispiel dist_total[r]:", list(dist_total.items())[:3]); print()
print("Charger pmax (kW):", pmax_ch); print()
print("e-Lkw batt_kwh:", batt_kwh); print()
print("Energiebedarf Beispiel (erste Route, e400/e600):")
first_r = R[0]
for e in cons_e.keys():
    print(" ", (first_r, e), "=", E_route_e[(first_r, e)], "kWh")
print()

print("Konstanten: P_grid_max =", P_grid_max, "| c_energy =", c_energy, "| c_toll =", c_toll); print()
print("Speicher: eta =", eta, "| soc_bat_min_frac =", soc_bat_min_frac); print()

Beispiel dist_total[r]: [('t-4', 250), ('t-5', 250), ('t-6', 250)]

Charger pmax (kW): {'Alpitronic-50': 50, 'Alpitronic-200': 200, 'Alpitronic-400': 400}

e-Lkw batt_kwh: {'eActros600': 621, 'eActros400': 414}

Energiebedarf Beispiel (erste Route, e400/e600):
  ('t-4', 'eActros600') = 275.0 kWh
  ('t-4', 'eActros400') = 262.5 kWh

Konstanten: P_grid_max = 500.0 | c_energy = 0.25 | c_toll = 0.34

Speicher: eta = 0.9899494936611666 | soc_bat_min_frac = 0.025000000000000022



### Entscheidungsvariablen

### Zielfunktion

### Restriktionen

### Berechnung des Ergebnisses